# 🎙️ Vedda ASR Model Training - Google Colab

Train a custom speech recognition model for Vedda language using Whisper on Google Colab with FREE GPU!

**⚠️ IMPORTANT:** Before running this notebook:
1. Click "Runtime" → "Change runtime type" → Select "GPU (T4)" → "Save"
2. Have your Vedda audio files ready (WAV format, 16kHz recommended)
3. Create a folder `vedda-asr-dataset` in your Google Drive with audio files and transcriptions

## 📋 Step 1: Setup Environment

Install all required dependencies

In [ ]:
# Check GPU availability
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ WARNING: GPU not available. Training will be very slow!")
    print("   Go to Runtime → Change runtime type → Select GPU")

In [ ]:
# Install required packages
print("🔧 Installing dependencies...")
!pip install -q openai-whisper transformers datasets evaluate jiwer librosa soundfile scikit-learn tqdm tensorboard accelerate
print("✅ Installation complete!")

## 📂 Step 2: Mount Google Drive

Connect your Google Drive to access your Vedda audio files

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')
print("✅ Google Drive mounted!")

# Check if dataset folder exists
dataset_dir = '/content/drive/MyDrive/vedda-asr-dataset'
if os.path.exists(dataset_dir):
    print(f"✅ Dataset found at: {dataset_dir}")
    print(f"\n📁 Contents:")
    for item in os.listdir(dataset_dir):
        print(f"   - {item}")
else:
    print(f"⚠️ Dataset folder not found at: {dataset_dir}")
    print(f"\n📋 Please create the folder in Google Drive and upload your audio files:")
    print(f"   1. Go to Google Drive")
    print(f"   2. Create folder 'vedda-asr-dataset'")
    print(f"   3. Upload audio files (WAV format)")
    print(f"   4. Create 'transcriptions.json' with transcriptions")

## 📝 Step 3: Prepare Dataset

Create training and testing datasets from your audio files

In [ ]:
import json
import os
from pathlib import Path

# Install noisereduce if not already installed
print("🔧 Installing audio processing tools...")
!pip install -q noisereduce
import noisereduce as nr

# Create project structure
project_dir = '/content/vedda-asr'
data_dir = os.path.join(project_dir, 'data')
processed_dir = os.path.join(data_dir, 'processed')
models_dir = os.path.join(project_dir, 'models')

os.makedirs(processed_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)

print(f"✅ Project structure created:")
print(f"   {project_dir}/")
print(f"   ├── data/")
print(f"   │   └── processed/")
print(f"   └── models/")

In [ ]:
import librosa
import soundfile as sf
import numpy as np
from tqdm import tqdm
import noisereduce as nr

def prepare_dataset():
    """
    Prepare dataset from audio files in Google Drive with noise cancellation

    Expected structure in Google Drive (vedda-asr-dataset/):
    - audio/
      - vedda_001.wav
      - vedda_002.wav
      - ...
    - transcriptions.json (format: {"vedda_001": "හෙලෝ", "vedda_002": "ආයුබෝවන්", ...})
    """

    dataset_dir = '/content/drive/MyDrive/vedda-asr-dataset'
    audio_dir = os.path.join(dataset_dir, 'audio')
    transcriptions_file = os.path.join(dataset_dir, 'transcriptions.json')

    # Check if files exist
    if not os.path.exists(audio_dir):
        print(f"❌ Audio directory not found: {audio_dir}")
        print(f"\n📋 Create this structure in Google Drive:")
        print(f"   vedda-asr-dataset/")
        print(f"   ├── audio/")
        print(f"   │   ├── vedda_001.wav")
        print(f"   │   ├── vedda_002.wav")
        print(f"   │   └── ...")
        print(f"   └── transcriptions.json")
        return None

    if not os.path.exists(transcriptions_file):
        print(f"❌ Transcriptions file not found: {transcriptions_file}")
        print(f"\n📋 Create transcriptions.json with format:")
        print(f'   {{"vedda_001": "හෙලෝ", "vedda_002": "ආයුබෝවන්"}}')
        return None

    # Load transcriptions
    with open(transcriptions_file, 'r', encoding='utf-8') as f:
        transcriptions = json.load(f)

    print(f"📂 Found {len(transcriptions)} transcriptions")

    # Process audio files
    processed_entries = []
    audio_files = [f for f in os.listdir(audio_dir) if f.lower().endswith(('.wav', '.mp3', '.ogg', '.flac'))]

    print(f"🔧 Processing {len(audio_files)} audio files with noise cancellation...")

    for audio_file in tqdm(audio_files):
        audio_path = os.path.join(audio_dir, audio_file)
        file_id = Path(audio_file).stem

        # Get transcription
        if file_id not in transcriptions:
            print(f"⚠️  Skipping {audio_file} (no transcription)")
            continue

        transcription = transcriptions[file_id]

        try:
            # Load audio
            audio, sr = librosa.load(audio_path, sr=None, mono=True)

            # Resample to 16kHz
            audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)

            # ===== NOISE CANCELLATION =====
            print(f"  🔇 Removing noise from {audio_file}...")
            audio = nr.reduce_noise(
                y=audio,
                sr=16000,
                stationary=True,          # Remove stationary background noise
                prop_decrease=1.0,        # Aggressiveness (0-1)
                chunk_size=600000,        # Memory-efficient chunking
                padding=30000,
                freq_mask_smooth_hz=500,  # Smooth frequency mask
                time_mask_smooth_ms=50    # Smooth time mask
            )

            # Normalize
            audio = librosa.util.normalize(audio)

            # Trim silence
            audio, _ = librosa.effects.trim(audio, top_db=25)

            # Pre-emphasis
            audio = librosa.effects.preemphasis(audio)

            # Final normalize
            audio = librosa.util.normalize(audio)

            duration = len(audio) / 16000
            if duration < 0.5:
                continue

            # Save processed audio
            output_path = os.path.join(processed_dir, audio_file)
            sf.write(output_path, audio, 16000)

            processed_entries.append({
                'audio_path': output_path,
                'transcription': transcription,
                'duration': duration,
                'noise_cancelled': True
            })

        except Exception as e:
            print(f"⚠️  Error processing {audio_file}: {e}")
            continue

    print(f"\n✅ Processed {len(processed_entries)} audio files (with noise cancellation)")

    total_duration = sum(e['duration'] for e in processed_entries)
    print(f"⏱️  Total duration: {total_duration/60:.1f} minutes")

    if total_duration < 600:  # Less than 10 minutes
        print(f"⚠️  WARNING: Dataset is small (1 hour recommended)")

    return processed_entries

# Prepare dataset
dataset = prepare_dataset()

if dataset is None:
    print("\n⚠️  Please set up your dataset first!")

In [ ]:
# Split dataset into train/test
if dataset:
    from sklearn.model_selection import train_test_split

    train_data, test_data = train_test_split(dataset, test_size=0.1, random_state=42)

    print(f"📊 Dataset split:")
    print(f"   Train: {len(train_data)} samples ({sum(e['duration'] for e in train_data)/60:.1f} min)")
    print(f"   Test:  {len(test_data)} samples ({sum(e['duration'] for e in test_data)/60:.1f} min)")

    # Save datasets
    train_file = os.path.join(data_dir, 'train_dataset.json')
    test_file = os.path.join(data_dir, 'test_dataset.json')

    with open(train_file, 'w', encoding='utf-8') as f:
        json.dump({'data': train_data}, f, ensure_ascii=False)

    with open(test_file, 'w', encoding='utf-8') as f:
        json.dump({'data': test_data}, f, ensure_ascii=False)

    print(f"\n✅ Datasets saved")

## 🎓 Step 4: Train Whisper Model

Fine-tune OpenAI Whisper on your Vedda data

In [ ]:
from transformers import (
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
from datasets import Dataset, Audio
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

print("✅ Data structures ready")

In [ ]:
# Load model and processor
print("ဒဢ Loading Whisper-small model...")

model_name = "openai/whisper-small"
processor = WhisperProcessor.from_pretrained(
    model_name,
    language="Sinhala",
    task="transcribe"
)

model = WhisperForConditionalGeneration.from_pretrained(model_name)

# Correct way to modify generation settings in newer transformers versions
model.generation_config.forced_decoder_ids = None
model.generation_config.suppress_tokens = []

print(f"✅ Model loaded: {model_name}")
print(f"ဒဠ Parameters: {model.num_parameters() / 1e6:.1f}M")

In [ ]:
# Load datasets
print("📂 Loading datasets...")

# Define paths (in case Step 3 cells weren't executed sequentially)
data_dir = '/content/vedda-asr/data'
train_file = os.path.join(data_dir, 'train_dataset.json')
test_file = os.path.join(data_dir, 'test_dataset.json')

# Check if dataset files exist
if not os.path.exists(train_file) or not os.path.exists(test_file):
    print(f"❌ Error: Dataset files not found!")
    print(f"   Train file: {train_file} ({'✅' if os.path.exists(train_file) else '❌'})")
    print(f"   Test file: {test_file} ({'✅' if os.path.exists(test_file) else '❌'})")
    print(f"\n📋 Please run Step 3 cells in order:")
    print(f"   1. 'Prepare dataset' cell")
    print(f"   2. 'Split dataset into train/test' cell")
else:
    with open(train_file, 'r', encoding='utf-8') as f:
        train_json = json.load(f)
    with open(test_file, 'r', encoding='utf-8') as f:
        test_json = json.load(f)

    train_dataset = Dataset.from_dict({
        'audio': [x['audio_path'] for x in train_json['data']],
        'transcription': [x['transcription'] for x in train_json['data']]
    })

    test_dataset = Dataset.from_dict({
        'audio': [x['audio_path'] for x in test_json['data']],
        'transcription': [x['transcription'] for x in test_json['data']]
    })

    train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000))
    test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=16000))

    print(f"✅ Train samples: {len(train_dataset)}")
    print(f"✅ Test samples: {len(test_dataset)}")

In [ ]:
# Prepare datasets
print("🔧 Preparing datasets for training...")

def prepare_dataset(batch):
    audio = batch["audio"]
    input_features = processor.feature_extractor(
        audio["array"],
        sampling_rate=audio["sampling_rate"]
    ).input_features[0]
    labels = processor.tokenizer(batch["transcription"]).input_ids
    return {
        "input_features": input_features,
        "labels": labels
    }

train_dataset_proc = train_dataset.map(
    prepare_dataset,
    remove_columns=train_dataset.column_names
)
test_dataset_proc = test_dataset.map(
    prepare_dataset,
    remove_columns=test_dataset.column_names
)

print(f"✅ Datasets prepared")

In [ ]:
# Training arguments
output_dir = os.path.join(models_dir, 'whisper-vedda')

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=50,
    num_train_epochs=10,
    eval_strategy="steps",
    eval_steps=50,
    save_steps=100,
    logging_steps=25,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
    save_total_limit=3,
    fp16=torch.cuda.is_available(),
    report_to=["tensorboard"],
    predict_with_generate=True,
    generation_max_length=225,
)

data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id
)

print(f"✅ Training configuration ready")
print(f"   Output: {output_dir}")
print(f"   Epochs: 10")
print(f"   Batch size: 8")

In [ ]:
# Create trainer
def compute_metrics(pred):
    from evaluate import load
    wer_metric = load("wer")
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_proc,
    eval_dataset=test_dataset_proc,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print(f"✅ Trainer ready")

In [ ]:
# Train model
print(f"\n{'='*60}")
print(f"🚀 STARTING TRAINING")
print(f"{'='*60}\n")

trainer.train()

print(f"\n{'='*60}")
print(f"✅ TRAINING COMPLETE")
print(f"{'='*60}")

## 📊 Step 5: Evaluate Model

In [ ]:
# Evaluate on test set
print("📊 Evaluating on test set...\n")

results = trainer.evaluate(test_dataset_proc)

print(f"\n{'='*60}")
print(f"📈 EVALUATION RESULTS")
print(f"{'='*60}")
print(f"\nWord Error Rate (WER): {results['eval_wer']*100:.2f}%")

if results['eval_wer'] < 0.10:
    quality = "🌟 Excellent (Production-ready)"
elif results['eval_wer'] < 0.20:
    quality = "✅ Good (Usable)"
elif results['eval_wer'] < 0.30:
    quality = "⚠️  Fair (Needs more data)"
else:
    quality = "❌ Poor (Collect more data)"

print(f"Quality: {quality}")
print(f"{'='*60}\n")

## 💾 Step 6: Save and Download Model

In [ ]:
# Save final model
final_dir = os.path.join(models_dir, 'whisper-vedda-final')
os.makedirs(final_dir, exist_ok=True)

print(f"💾 Saving final model...")
model.save_pretrained(final_dir)
processor.save_pretrained(final_dir)

print(f"✅ Model saved to: {final_dir}")

In [ ]:
# Create ZIP file for download
import shutil

print(f"ခူ Creating download package...")
zip_path = os.path.join(project_dir, 'vedda-asr-model')
shutil.make_archive(zip_path, 'zip', final_dir)

zip_file = zip_path + '.zip'
size_mb = os.path.getsize(zip_file) / 1e6

print(f"\n✅ Model packaged for download!")
print(f"   File: vedda-asr-model.zip")
print(f"   Size: {size_mb:.1f} MB")

print(f"\nေဳ The file is available in Colab's file browser (left panel)")
print(f"   Right-click → Download")

## 🧪 Step 7: Test Trained Model (Optional)

In [ ]:
# Test model on a sample audio
import librosa

print("🎤 Testing trained model...\n")

# Get first test sample
test_sample = test_json['data'][0]
audio_path = test_sample['audio_path']
reference = test_sample['transcription']

# Load audio
audio, sr = librosa.load(audio_path, sr=16000, mono=True)

# Process with processor
input_features = processor(
    audio,
    sampling_rate=16000,
    return_tensors="pt"
).input_features

# Generate prediction
with torch.no_grad():
    predicted_ids = model.generate(input_features)

prediction = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

print(f"📝 Sample Test Result:")
print(f"\n   Reference:  {reference}")
print(f"   Predicted:  {prediction}")
print(f"   Match: {'✅ YES' if reference == prediction else '❌ NO'}")

## 📋 Summary

### What you learned:
1. ✅ Set up GPU environment in Colab
2. ✅ Loaded Vedda audio data from Google Drive
3. ✅ Prepared dataset for training
4. ✅ Fine-tuned Whisper model on Vedda speech
5. ✅ Evaluated model performance (WER)
6. ✅ Saved and packaged trained model

### Next steps:
1. Download the trained model (vedda-asr-model.zip)
2. Extract it locally
3. Integrate with your speech service
4. For better accuracy: Collect more data and retrain

### Performance improvements:
- **More data:** 1 hour → Better accuracy
- **More epochs:** 10 → 15 or 20
- **Larger model:** small → base or medium
- **Lower learning rate:** 1e-5 → 5e-6

---

**Happy training! 🚀**